# **PRÁCTICA 4:** Resolución aproximada de ecuaciones mediante los métodos de Horner y Sturm

## 1. Método de Horner

Implementar el Método de Horner para que, dado un polinomio $P(x)$ de
cualquier grado y un punto $x_0 \in \mathbb{R}$, se determine otro polinomio $Q(x)$ tal que

$$ P(x) = (x-x_0) \: Q(x) + P(x_0) $$

In [1]:
import numpy as np
from sympy import *

In [2]:
def Horner(P,x_0):
    
    #Variable donde guardar el polinomio
    result = []
    
    #Insertar el valor del polinomio con el x_0
    result.append(P.subs(x,x_0))
    
    #Obtener el coeficiente de mayor grado
    mayor_coef = P.coeff(x,degree(P))
    
    #Insertar el coeficiente con mayor grado del polinomio
    result.append(mayor_coef);
    
    #bucle con el tamaño de los siguientes grados del polinomio
    for i in range(1,degree(P)):
        
        #Insertar en el resultado la suma del siguiente coeficiente de mayor grado y el anterior valor multiplicado por el x_0
        result.append(P.coeff(x,degree(P)-i) + result[-1]*x_0)
    
    #Devolver todos los elementos excepto eñ primer elemento (polinomio inicial)
    return result[1:]

Utilizarla en:

$$ P(x) = 5x^3+7x^2+2x+3$$ con $x_0=0$ y $x_0 = 3$

In [3]:
#Substituir simbolo x
x = Symbol('x')

#Declarar polinomio y x_0
P = 5*(x**3) + 7*(x**2)+ 2*x +3
x_0 = 0

#Aplicar Horner
Q = Horner(P,0)

#Comprobación
P = np.array([5, 7, 2, 3])
Q = np.array(Q)
np.polyadd(np.polymul(np.array([1, -x_0]), Q), np.polyval(P, x_0)) == P

array([ True,  True,  True,  True])

## 2. Método de Sturm

1. Implementar el Método de Sturm para, dado un polinomio $P(x)$, **construir la secuencia de polinomios de Sturm**. Hacer una función que devuelva una lista de polinomios (ndarrays).

In [4]:
def SturmSequence(P):
    
    #Ponerlo en forma de polinomio
    P = Poly(P,x)
    
    #Declarar variable para guardar la secuencia de polinomios
    seq = []
    
    #Insertar primer polinomio en la secuencia de polinomios
    seq.append(P)
    
    #Calcular primera derivada del polinomio inicial
    primera_derivada = diff(P,x)
    
    #Insertar primera derivada en la secuencia de polinomios
    seq.append(primera_derivada)
    
    #Bucle con el número de grados del polinomio -1
    for i in range(degree(P)-1):
        
        #Calular siguientes poinomios de la secuencia con landa(-1) por el resto de los anterioriores dos polinomios de la secuencia
        siguiente_polinomio = -(seq[-2] % seq[-1])
        
        #Insertar el siguiente polinomio en la secuencia
        seq.append(siguiente_polinomio)
        
    return seq

In [5]:
#Comprobacion

#Definir polinomio
P = x**3 - 9*x**2 + 24*x -36

#Aplicar Sturm
seq = SturmSequence(P)
seq

[Poly(x**3 - 9*x**2 + 24*x - 36, x, domain='ZZ'),
 Poly(3*x**2 - 18*x + 24, x, domain='ZZ'),
 Poly(2*x + 12, x, domain='ZZ'),
 Poly(-240, x, domain='ZZ')]

2. Complementar la implementación del apartado 1, para, utilizando el Método de Sturm, **hallar el número de raíces positivas y negativas** de $P(x)$ en un intervalo dado. Hacer una función que devuelva los valores `pos` y `neg`.

NOTA: Asumir que todas las raíces son simples. 

In [6]:
def numberOfRoots(P, min, max):
    
    #Definir variables donde iran los valores maximos y minimos
    valor_max=[]
    valor_min=[]
    
    #Poner P en forma de polinomio
    P = Poly(P,x)
    
    #Aplicar Sturm
    P = SturmSequence(P) 
    
    #Evaluar cada polinomio en cada exremo de un intervalo amplio
    for i, polinomio in enumerate(P):
        
        #Calcular el valor del polinomio en el punto mínimo
        minimo = polinomio.subs(x, min)
        
        #Insertar el valor minimo
        valor_min.append(minimo)
        
        #Calcular el valor del polinomio en el punto máximo
        maximo = polinomio.subs(x, max)
        
        #Insertar el valor máximo
        valor_max.append(maximo)
        
        #Variables para indicar cambio de signo con los valores maximos y minoimos
        cambios_signo_min = 0
        cambios_signo_max = 0
        
        #Bucle con el numero de valores máximos y mínimos
        #Empezar desde 1 para poder comparar el segundo elemento(1) con el primer elemento(0)
        for i in range(1, len(valor_max)):
            
            #Comprobar si se produce un cambio de signo
            if((valor_min[i] >= 0 and valor_min[i-1] < 0) or (valor_min[i] < 0 and valor_min[i-1] >= 0)):
                
                #Si se produce el cambio de signo se aumenta en 1 el contador de cambio de signo
                cambios_signo_min += 1;
            
            #Comprobar si se produce un cambio de signo
            if((valor_max[i] >= 0 and valor_max[i-1] < 0 ) or (valor_max[i] < 0 and valor_max[i-1] >= 0)):
                
                #Si se produce el cambio de signo se aumenta en 1 el contador de cambio de signo
                cambios_signo_max += 1;
        
        #Obtener el número de raíces
        raices = np.abs(cambios_signo_max - cambios_signo_min)
    
    return (raices, cambios_signo_min, cambios_signo_max)

In [7]:
#Comprobacion

#Definir polinomio
P = x**3 - 9*x**2 + 24*x -36

#Aplicar numberOfRoots
raices, pos, neg= numberOfRoots(P, -500 , 500)
raices, pos, neg

(1, 2, 1)

3. Utilizar la implementación del apartado 2 para **separar las raíces de un
polinomio** dado (es decir, encontrar intervalos que contengan una y solamente
una raíz). Hacer una función que devuelva una lista de intervalos.

NOTA: Asumir que todas las raíces del polinomio son reales y simples.


In [8]:
def SturmIntervals(P, min, max):
  
    #Definir variable para guardar los intervalos
    intervals = []
    
    #Aplicar numberOfRoots
    raices, pos, neg = numberOfRoots(P, min, max)
    
    #Crear posibles intervalos de 0.1
    intervalos = np.arange(min,max,0.1)
    
    i = 0
    
    #Mientras queden raices posibles
    n_raices = raices
    while n_raices > 0:
        
        #Para cada intervalo aplicar numberOfRoots
        raices, pos, neg = numberOfRoots(P, intervalos[i], intervalos[i+1])
        
        #Para cada intervalo comprobar si existe una raiz en ese intervalo
        if(raices == 1):
            
            #Si existe insertar el intervalo en la secuencia de intervalos
            intervals.append((intervalos[i], intervalos[i+1]))
            
            #Restar una raiz ya encontrada
            n_raices -=1
        
        #Incrementar el intervalo
        i+=1
             
    return intervals

Calcular los intervalos de separación de las raíces de los siguientes polinomios:

$$P_1(x) = x^3-9x^2+24x-36$$
$$P_2(x) = x^6 - 9.5x^5 + 18x^4 + 33.5x^3 - 97x^2 + 18x +36$$

In [9]:
#Cálculo

#Declarar polinomio
P1 = x**3-9*x**2+24*x-36

#Aplicar SturmIntervals
intervals1 = SturmIntervals(P1,-100,100)
print('Intervalos de P1: {}'.format(intervals1))

#Declarar polinomio
P2 = x**6 - 9.5*x**5 + 18*x**4 + 33.5*x**3 - 97*x**2 + 18*x +36

#Aplicar SturmIntervals
intervals2 = SturmIntervals(P2,-100,100)
print('Intervalos de P2: {}'.format(intervals2))

Intervalos de P1: [(5.999999999993975, 6.099999999993969)]
Intervalos de P2: [(-2.0000000000055707, -1.9000000000055763), (-0.5000000000056559, -0.4000000000056616), (0.9999999999942588, 1.0999999999942531), (1.999999999994202, 2.0999999999941963), (2.999999999994145, 3.0999999999941394), (5.999999999993975, 6.099999999993969)]
